<a href="https://colab.research.google.com/github/mininamaria/WatchThoseHands/blob/main/TimeOffsetCalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description

There are timestamps in video stimulus (retelling)

The file `benchmark.tsv`contains timestamps in video (retelling): the start and end of all stimulus and control *shape/manner* gestures.

As there are 15 participants, and each of them watched the video twice, there are **30 recordings**, and adding all the events in Pupil Cloud is extremely time-consuming. Due to technical conditions, recording start was not syncrinised with the beginning of the video.

This code reads `benchmark.tsv` and `offsets.csv`, the latter contains offsets for each recording - those were noted manually. After calculating the time of beginning and end of the gestures, a separate TSV file is created for every wearer-view combination (e.g. `G001-view_1.tsv`). The rows contain the timestamps and corresponding gesture IDs.

UPDATE: as Pupil Cloud accepts time in seconds (float) format, the TSV now looks like this:

```
seconds	timestamp	event
14.885	0:00:14.885	01-mY.start
15.86	0:00:15.860	01-mY.end
28.51	0:00:28.510	02-sN.start
30.11	0:00:30.110	02-sN.end
```

The timestamp column is left here for the sake of readability.

## Order

1. Open `benchmark.tsv` as pd.DataFrame
2. Read lines of `offsets.tsv`.
3. From each line get `wearer`, `view`, `offset`
4. Create file `wearer-view.tsv`
5. Iterate through the rows of `benchmark`
6. Get `timestamp_id`, `start`, `end`
7. Write into `wearer-view.tsv` the following:
  - `timestamp_id`.start . . . `start` + `offset`
  - `timestamp_id`.end . . . `end` + `offset`


## Code

In [ ]:
import pandas as pd

benchmark = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/Курсовая 2025: следим за руками/data/eyetracker/timestamps/benchmark.tsv", sep = "\t"))
benchmark.head(5)

,gesture_id,phrase,gesture_type,gesture,match,description,start,end,timestamp_id
0,1,влетающего в комнату для борьбы.,manner,мах рукой 1 сбоку в центр,1.0,влетает,0:00:05.775,0:00:06.750,01-mY
1,2,Этот панда значительно ниже волка,shape,"поднимает обе руки на уровень подбородка, анаф...",0.0,выше,0:00:19.400,0:00:21.000,02-sN
2,3,переворачивает туда-сюда.,manner,переворачивает,1.0,переворачивает,0:00:22.830,0:00:24.155,03-mY
3,4,"Панда делает волку болевой захват, тот стучит ...",manner,медленно гладит одной рукой горизонтальную пов...,0.0,гладит пол,0:00:28.700,0:00:31.000,04-mN
4,5,"Волк видит это, вырывается и проползает под ко...",manner,собранной в щепоть рукой совершает движения по...,0.0,пропрыгивает,0:00:36.225,0:00:37.830,05-mN


In [ ]:
for index, row in benchmark.iterrows():
    print(f"{row['timestamp_id']}.start\t{row['start']}")
    print(f"{row['timestamp_id']}.end\t{row['end']}")

In [38]:
from datetime import timedelta

def parse_time_offset(time_str):
    h, m, s_ms = time_str.split(":")
    s, ms = s_ms.split(".")
    return timedelta(hours=int(h), minutes=int(m), seconds=int(s), milliseconds=int(ms))

def add_time_offsets(t1_str, t2_str):
    delta = parse_time_offset(t1_str) + parse_time_offset(t2_str)
    total_seconds = int(delta.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    milliseconds = delta.microseconds // 1000
    return f"{delta.total_seconds()}\t{hours}:{minutes:02}:{seconds:02}.{milliseconds:03}"

# Example
print(add_time_offsets("0:00:55.775", "0:00:09.110"))

64.885	0:01:04.885


In [ ]:
def wearer_view_offsets_log(wv_offset: str):
  result = ""
  for index, row in benchmark.iterrows():
    gesture_start = add_time_offsets(row['start'], wv_offset)
    gesture_end = add_time_offsets(row['end'], wv_offset)
    result += (f"{gesture_start}\t{row['timestamp_id']}.start\n\
                {gesture_end}\t{row['timestamp_id']}.end\n")
  return result

In [39]:
with open("/content/drive/MyDrive/Курсовая 2025: следим за руками/data/eyetracker/timestamps/offsets.csv", 'r') as o:
  for l in o.readlines():
    wearer, view, offset = l.split(",")
    flnm = f"/content/drive/MyDrive/Курсовая 2025: следим за руками/data/eyetracker/timestamps/{wearer}-{view}.tsv"
    with open(flnm,"w") as wearer_view_file:
      wearer_view_file.write("seconds\ttimestamp\tevent\n")
      wearer_view_file.write(wearer_view_offsets_log(offset))
      print(flnm, "created")

/content/drive/MyDrive/Курсовая 2025: следим за руками/data/eyetracker/timestamps/﻿G001-view_1.tsv created
